<a href="https://colab.research.google.com/gist/maclandrol/00f78b29c1dc1b960a730999982c6143/08_TorchXRayVision_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Enseignant:** Emmanuel Noutahi, PhD

# Tutorial 3 : Segmentation Anatomique avec TorchXRayVision
# Tutorial 3: Anatomical Segmentation with TorchXRayVision

---

## 📚 Contexte Médical / Medical Context

### Pour les Étudiants en Médecine / For Medical Students
La **segmentation anatomique** en radiologie consiste à identifier et délimiter précisément les structures anatomiques dans une image radiographique. Cette technique est fondamentale pour :
- **Anatomie** : Comprendre la morphologie et les rapports anatomiques
- **Physiologie** : Analyser les fonctions des différentes structures
- **Radiologie** : Interpréter les images médicales avec précision

### Pour les Praticiens / For Practitioners
- **Chirurgiens** : Planification pré-opératoire et guidage per-opératoire
- **Médecins Généralistes** : Aide au diagnostic et orientation des patients
- **Enseignants d'Anatomie** : Outil pédagogique pour l'enseignement interactif

---

## 🎯 Objectifs d'Apprentissage / Learning Objectives

À la fin de ce tutoriel, vous serez capable de :
1. Comprendre les principes de la segmentation anatomique
2. Utiliser les modèles de segmentation de TorchXRayVision
3. Segmenter les principales structures thoraciques
4. Visualiser et analyser les résultats de segmentation
5. Interpréter les résultats dans un contexte clinique

---

## 📋 Prérequis / Prerequisites

Ce tutoriel fait suite aux **Tutoriels 1 et 2**. Assurez-vous d'avoir complété :
- Tutorial 1: Introduction to TorchXRayVision
- Tutorial 2: Chest X-ray Classification

---

## 🔧 Installation et Configuration / Setup

In [ ]:
# Installation des bibliothèques nécessaires / Install required libraries
!pip install torchxrayvision
!pip install torch torchvision
!pip install matplotlib seaborn
!pip install numpy pandas
!pip install scikit-image
!pip install opencv-python

print("✅ Installation terminée / Installation completed")

In [ ]:
# Import des bibliothèques / Import libraries
import torch
import torch.nn as nn
import torchxrayvision as xrv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from skimage import measure, morphology
from google.colab import files
import io
import warnings
warnings.filterwarnings('ignore')

# Configuration de l'affichage / Display configuration
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("📚 Bibliothèques importées avec succès / Libraries imported successfully")
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🏥 TorchXRayVision version: {xrv.__version__}")

# Vérification du GPU / GPU check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"💻 Device utilisé / Device used: {device}")

## 🧠 Théorie : Segmentation Anatomique / Theory: Anatomical Segmentation

### Qu'est-ce que la Segmentation ? / What is Segmentation?

La **segmentation** consiste à diviser une image en régions homogènes correspondant à des structures anatomiques spécifiques :

- **Segmentation binaire** : Sépare une structure du reste (ex: poumons vs fond)
- **Segmentation multi-classes** : Identifie plusieurs structures simultanément

### Structures Anatomiques Segmentables / Segmentable Anatomical Structures

Dans une radiographie thoracique, nous pouvons segmenter :
- **Poumons** (lung fields)
- **Cœur** (cardiac silhouette)
- **Côtes** (ribs)
- **Clavicules** (clavicles)
- **Diaphragme** (diaphragm)

### Applications Cliniques / Clinical Applications

- **Mesure automatique** : Volume pulmonaire, ratio cardio-thoracique
- **Détection d'anomalies** : Localisation précise des pathologies
- **Suivi longitudinal** : Évolution des structures dans le temps
- **Planification thérapeutique** : Guidage des interventions

## 🏥 Modèles de Segmentation Disponibles / Available Segmentation Models

TorchXRayVision propose plusieurs modèles de segmentation spécialisés :

In [ ]:
# Chargement des modèles de segmentation / Load segmentation models
print("🔄 Chargement des modèles de segmentation...")
print("🔄 Loading segmentation models...")

# Modèle pour la segmentation des poumons / Lung segmentation model
try:
    # Utilisation du modèle lung segmentation de TorchXRayVision
    lung_model = xrv.baseline_models.chestx_det.PSPNet()
    print("✅ Modèle de segmentation pulmonaire chargé / Lung segmentation model loaded")
except Exception as e:
    print(f"⚠️ Erreur de chargement: {e}")
    # Alternative : utiliser un modèle de classification pour approximer
    lung_model = xrv.models.DenseNet(weights="densenet121-res224-all")
    print("✅ Modèle alternatif chargé / Alternative model loaded")

lung_model.to(device)
lung_model.eval()

print(f"📊 Modèle sur le device: {next(lung_model.parameters()).device}")

## 📤 Chargement d'Images / Image Upload

### Option 1 : Utiliser une Image d'Exemple / Use Sample Image

In [ ]:
# Chargement d'une image d'exemple depuis TorchXRayVision / Load sample image
print("📥 Chargement d'une image d'exemple...")
print("📥 Loading sample image...")

# Utilisation du dataset MIMIC pour obtenir une image d'exemple
try:
    # Charger un échantillon depuis les données de test
    sample_data = xrv.datasets.MIMIC_Dataset(
        imgpath=".",
        csvpath=".",
        metacsvpath=".",
        views=["PA", "AP"],
        transform=xrv.datasets.XRayResizer(224)
    )
    
    # Si pas d'accès aux données, utiliser une image générée
    print("ℹ️ Utilisation d'une image simulée pour la démonstration")
    print("ℹ️ Using simulated image for demonstration")
    
    # Créer une image simulée de radiographie thoracique
    img_array = np.random.rand(224, 224) * 0.3 + 0.4  # Image grisâtre
    
    # Ajouter des structures simulées
    # Poumons (zones plus sombres)
    img_array[50:180, 30:100] *= 0.7  # Poumon gauche
    img_array[50:180, 124:194] *= 0.7  # Poumon droit
    
    # Cœur (zone centrale)
    img_array[120:180, 90:134] *= 1.3
    
    original_image = Image.fromarray((img_array * 255).astype(np.uint8))
    
except Exception as e:
    print(f"⚠️ Erreur: {e}")
    # Créer une image de test simple
    img_array = np.random.rand(224, 224)
    original_image = Image.fromarray((img_array * 255).astype(np.uint8))

print("✅ Image d'exemple chargée / Sample image loaded")
print(f"📏 Taille de l'image / Image size: {original_image.size}")

### Option 2 : Charger Votre Propre Image / Upload Your Own Image

In [ ]:
# Option pour charger une image personnalisée / Option to upload custom image
print("📤 Voulez-vous charger votre propre image ?")
print("📤 Would you like to upload your own image?")
print("")
print("Cliquez sur 'Choisir des fichiers' ci-dessous pour charger une radiographie thoracique")
print("Click 'Choose Files' below to upload a chest X-ray")

# Interface de chargement
uploaded = files.upload()

# Traitement de l'image chargée
if uploaded:
    # Prendre le premier fichier chargé
    filename = list(uploaded.keys())[0]
    print(f"📁 Fichier chargé / File uploaded: {filename}")
    
    # Charger et traiter l'image
    original_image = Image.open(io.BytesIO(uploaded[filename]))
    
    # Convertir en niveaux de gris si nécessaire
    if original_image.mode != 'L':
        original_image = original_image.convert('L')
    
    print("✅ Image personnalisée chargée / Custom image loaded")
    print(f"📏 Taille originale / Original size: {original_image.size}")
else:
    print("ℹ️ Utilisation de l'image d'exemple / Using sample image")

# Affichage de l'image originale
plt.figure(figsize=(8, 8))
plt.imshow(original_image, cmap='gray')
plt.title("Image Originale / Original Image")
plt.axis('off')
plt.show()

## 🔧 Préparation de l'Image / Image Preprocessing

Pour une segmentation optimale, nous devons préparer l'image selon les standards TorchXRayVision :

In [ ]:
def preprocess_for_segmentation(image):
    """
    Préparation de l'image pour la segmentation
    Preprocessing image for segmentation
    """
    print("🔧 Préparation de l'image pour la segmentation...")
    print("🔧 Preprocessing image for segmentation...")
    
    # Convertir en array numpy
    img_array = np.array(image)
    
    # Redimensionner à 224x224 (standard TorchXRayVision)
    if img_array.shape != (224, 224):
        img_resized = cv2.resize(img_array, (224, 224))
    else:
        img_resized = img_array
    
    # Normalisation pour TorchXRayVision
    # Convertir en float et normaliser entre 0-1
    img_normalized = img_resized.astype(np.float32) / 255.0
    
    # Normalisation Z-score (moyenne 0, écart-type 1)
    mean = np.mean(img_normalized)
    std = np.std(img_normalized)
    img_zscore = (img_normalized - mean) / (std + 1e-8)
    
    # Convertir en tensor PyTorch
    img_tensor = torch.FloatTensor(img_zscore)
    img_tensor = img_tensor.unsqueeze(0).unsqueeze(0)  # Ajouter batch et canal
    
    print(f"📏 Taille finale du tensor / Final tensor shape: {img_tensor.shape}")
    print(f"📊 Valeurs min/max / Min/max values: {img_tensor.min():.3f} / {img_tensor.max():.3f}")
    
    return img_tensor, img_resized, img_normalized

# Appliquer le preprocessing
img_tensor, img_resized, img_normalized = preprocess_for_segmentation(original_image)
img_tensor = img_tensor.to(device)

# Visualisation des étapes de preprocessing
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(original_image, cmap='gray')
axes[0].set_title('Image Originale / Original')
axes[0].axis('off')

axes[1].imshow(img_resized, cmap='gray')
axes[1].set_title('Redimensionnée / Resized (224x224)')
axes[1].axis('off')

axes[2].imshow(img_normalized, cmap='gray')
axes[2].set_title('Normalisée / Normalized')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Préparation terminée / Preprocessing completed")

## 🫁 Segmentation des Poumons / Lung Segmentation

### Méthode 1 : Segmentation Basée sur l'Intensité / Intensity-Based Segmentation

In [ ]:
def segment_lungs_intensity(image_array):
    """
    Segmentation des poumons basée sur l'intensité des pixels
    Lung segmentation based on pixel intensity
    """
    print("🫁 Segmentation des poumons par intensité...")
    print("🫁 Lung segmentation by intensity...")
    
    # Appliquer un seuillage pour identifier les zones pulmonaires
    # Les poumons apparaissent plus sombres (valeurs plus faibles)
    threshold = np.percentile(image_array, 40)  # Seuil au 40e percentile
    
    # Créer le masque binaire
    lung_mask = image_array < threshold
    
    # Nettoyage morphologique
    # Éliminer les petites zones
    lung_mask = morphology.remove_small_objects(lung_mask, min_size=500)
    
    # Combler les petits trous
    lung_mask = morphology.remove_small_holes(lung_mask, area_threshold=300)
    
    # Étiquetage des composantes connexes pour séparer les poumons
    labeled_mask = measure.label(lung_mask)
    
    # Garder les 2 plus grandes composantes (poumons gauche et droit)
    props = measure.regionprops(labeled_mask)
    if len(props) >= 2:
        # Trier par aire décroissante
        props_sorted = sorted(props, key=lambda x: x.area, reverse=True)
        
        # Créer le masque final avec les 2 plus grandes régions
        final_mask = np.zeros_like(labeled_mask, dtype=bool)
        for i in range(min(2, len(props_sorted))):
            final_mask[labeled_mask == props_sorted[i].label] = True
        
        lung_mask = final_mask
    
    print(f"📊 Aire pulmonaire segmentée: {np.sum(lung_mask)} pixels")
    print(f"📊 Pourcentage de l'image: {np.sum(lung_mask)/lung_mask.size*100:.1f}%")
    
    return lung_mask

# Appliquer la segmentation par intensité
lung_mask_intensity = segment_lungs_intensity(img_normalized)

# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Image originale
axes[0].imshow(img_normalized, cmap='gray')
axes[0].set_title('Image Originale / Original Image')
axes[0].axis('off')

# Masque de segmentation
axes[1].imshow(lung_mask_intensity, cmap='Blues')
axes[1].set_title('Masque Pulmonaire / Lung Mask')
axes[1].axis('off')

# Superposition
overlay = img_normalized.copy()
overlay[lung_mask_intensity] = np.maximum(overlay[lung_mask_intensity], 0.7)
axes[2].imshow(overlay, cmap='gray')
axes[2].contour(lung_mask_intensity, colors='red', linewidths=2)
axes[2].set_title('Superposition / Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Segmentation par intensité terminée / Intensity segmentation completed")

### Méthode 2 : Segmentation Avancée avec Gradient / Advanced Gradient-Based Segmentation

In [ ]:
def segment_lungs_gradient(image_array):
    """
    Segmentation avancée utilisant les gradients et contours
    Advanced segmentation using gradients and contours
    """
    print("🔍 Segmentation avancée par gradients...")
    print("🔍 Advanced gradient-based segmentation...")
    
    # Convertir en uint8 pour OpenCV
    img_uint8 = (image_array * 255).astype(np.uint8)
    
    # Appliquer un filtre gaussien pour réduire le bruit
    img_smooth = cv2.GaussianBlur(img_uint8, (5, 5), 0)
    
    # Détection des contours avec Canny
    edges = cv2.Canny(img_smooth, 50, 150)
    
    # Fermeture morphologique pour connecter les contours
    kernel = np.ones((3, 3), np.uint8)
    edges_closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # Remplissage des contours fermés
    contours, _ = cv2.findContours(edges_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Créer le masque en remplissant les contours
    mask = np.zeros_like(img_uint8)
    
    # Filtrer les contours par aire et position
    lung_contours = []
    h, w = img_uint8.shape
    
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, cw, ch = cv2.boundingRect(contour)
        
        # Critères pour les poumons :
        # - Aire suffisante
        # - Position dans la partie supérieure/moyenne de l'image
        # - Largeur raisonnable
        if (area > 1000 and y < h * 0.8 and cw > 20 and ch > 30):
            lung_contours.append(contour)
    
    # Dessiner et remplir les contours pulmonaires
    if lung_contours:
        cv2.drawContours(mask, lung_contours, -1, 255, -1)
    
    # Convertir en booléen
    lung_mask = mask > 0
    
    # Post-traitement morphologique
    lung_mask = morphology.remove_small_objects(lung_mask, min_size=300)
    lung_mask = morphology.remove_small_holes(lung_mask, area_threshold=200)
    
    print(f"📊 Nombre de contours pulmonaires détectés: {len(lung_contours)}")
    print(f"📊 Aire pulmonaire segmentée: {np.sum(lung_mask)} pixels")
    
    return lung_mask, edges

# Appliquer la segmentation par gradients
lung_mask_gradient, edge_map = segment_lungs_gradient(img_normalized)

# Visualisation comparative
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Ligne 1 : Méthode par intensité
axes[0, 0].imshow(img_normalized, cmap='gray')
axes[0, 0].set_title('Image Originale / Original')
axes[0, 0].axis('off')

axes[0, 1].imshow(lung_mask_intensity, cmap='Blues')
axes[0, 1].set_title('Segmentation par Intensité\nIntensity-Based')
axes[0, 1].axis('off')

overlay1 = img_normalized.copy()
axes[0, 2].imshow(overlay1, cmap='gray')
axes[0, 2].contour(lung_mask_intensity, colors='red', linewidths=2)
axes[0, 2].set_title('Résultat Intensité / Intensity Result')
axes[0, 2].axis('off')

# Ligne 2 : Méthode par gradients
axes[1, 0].imshow(edge_map, cmap='gray')
axes[1, 0].set_title('Détection de Contours / Edge Detection')
axes[1, 0].axis('off')

axes[1, 1].imshow(lung_mask_gradient, cmap='Greens')
axes[1, 1].set_title('Segmentation par Gradients\nGradient-Based')
axes[1, 1].axis('off')

overlay2 = img_normalized.copy()
axes[1, 2].imshow(overlay2, cmap='gray')
axes[1, 2].contour(lung_mask_gradient, colors='green', linewidths=2)
axes[1, 2].set_title('Résultat Gradients / Gradient Result')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("✅ Segmentation par gradients terminée / Gradient segmentation completed")

## ❤️ Segmentation Cardiaque / Cardiac Segmentation

La **silhouette cardiaque** est une structure importante à segmenter pour :
- Mesurer le **ratio cardio-thoracique** (normal < 0.5)
- Détecter une **cardiomégalie** 
- Évaluer la morphologie cardiaque

In [ ]:
def segment_cardiac_silhouette(image_array):
    """
    Segmentation de la silhouette cardiaque
    Cardiac silhouette segmentation
    """
    print("❤️ Segmentation de la silhouette cardiaque...")
    print("❤️ Cardiac silhouette segmentation...")
    
    h, w = image_array.shape
    
    # Zone d'intérêt pour le cœur (médiane, partie inférieure)
    roi_x_start = int(w * 0.35)
    roi_x_end = int(w * 0.65)
    roi_y_start = int(h * 0.45)
    roi_y_end = int(h * 0.85)
    
    # Extraire la ROI
    roi = image_array[roi_y_start:roi_y_end, roi_x_start:roi_x_end]
    
    # Le cœur apparaît généralement plus dense (valeurs plus élevées)
    threshold = np.percentile(roi, 75)  # Seuil au 75e percentile
    
    # Créer le masque cardiaque initial
    cardiac_mask_roi = roi > threshold
    
    # Nettoyage morphologique
    cardiac_mask_roi = morphology.remove_small_objects(cardiac_mask_roi, min_size=100)
    cardiac_mask_roi = morphology.remove_small_holes(cardiac_mask_roi, area_threshold=50)
    
    # Appliquer une fermeture pour connecter les parties
    selem = morphology.disk(3)
    cardiac_mask_roi = morphology.binary_closing(cardiac_mask_roi, selem)
    
    # Replacer dans l'image complète
    cardiac_mask = np.zeros_like(image_array, dtype=bool)
    cardiac_mask[roi_y_start:roi_y_end, roi_x_start:roi_x_end] = cardiac_mask_roi
    
    # Calculer les métriques
    cardiac_area = np.sum(cardiac_mask)
    total_area = h * w
    cardiac_ratio = cardiac_area / total_area
    
    print(f"📊 Aire cardiaque segmentée: {cardiac_area} pixels")
    print(f"📊 Ratio cardiaque/image: {cardiac_ratio:.3f}")
    
    return cardiac_mask, (roi_x_start, roi_y_start, roi_x_end, roi_y_end)

# Appliquer la segmentation cardiaque
cardiac_mask, roi_coords = segment_cardiac_silhouette(img_normalized)

# Visualisation
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

# Image originale avec ROI
axes[0].imshow(img_normalized, cmap='gray')
rect = plt.Rectangle((roi_coords[0], roi_coords[1]), 
                    roi_coords[2]-roi_coords[0], roi_coords[3]-roi_coords[1],
                    linewidth=2, edgecolor='yellow', facecolor='none')
axes[0].add_patch(rect)
axes[0].set_title('ROI Cardiaque / Cardiac ROI')
axes[0].axis('off')

# Masque cardiaque
axes[1].imshow(cardiac_mask, cmap='Reds')
axes[1].set_title('Masque Cardiaque / Cardiac Mask')
axes[1].axis('off')

# Superposition cardiaque
axes[2].imshow(img_normalized, cmap='gray')
axes[2].contour(cardiac_mask, colors='red', linewidths=2)
axes[2].set_title('Silhouette Cardiaque / Cardiac Silhouette')
axes[2].axis('off')

# Segmentation combinée (poumons + cœur)
axes[3].imshow(img_normalized, cmap='gray')
axes[3].contour(lung_mask_gradient, colors='blue', linewidths=2, linestyles='--')
axes[3].contour(cardiac_mask, colors='red', linewidths=2)
axes[3].set_title('Segmentation Combinée\nCombined Segmentation')
axes[3].axis('off')

plt.tight_layout()
plt.show()

print("✅ Segmentation cardiaque terminée / Cardiac segmentation completed")

## 📊 Analyse Quantitative / Quantitative Analysis

### Calcul des Métriques Cliniques / Clinical Metrics Calculation

In [ ]:
def calculate_clinical_metrics(lung_mask, cardiac_mask, image_shape):
    """
    Calcul des métriques cliniques importantes
    Calculate important clinical metrics
    """
    print("📊 Calcul des métriques cliniques...")
    print("📊 Calculating clinical metrics...")
    
    h, w = image_shape
    
    # 1. Aires en pixels
    lung_area = np.sum(lung_mask)
    cardiac_area = np.sum(cardiac_mask)
    total_area = h * w
    
    # 2. Ratio cardio-thoracique (CTR)
    # Diamètre cardiaque / diamètre thoracique
    
    # Calculer les diamètres
    cardiac_props = measure.regionprops(cardiac_mask.astype(int))
    lung_props = measure.regionprops(lung_mask.astype(int))
    
    if cardiac_props and lung_props:
        # Diamètre cardiaque (largeur de la bounding box)
        cardiac_bbox = cardiac_props[0].bbox
        cardiac_width = cardiac_bbox[3] - cardiac_bbox[1]  # xmax - xmin
        
        # Diamètre thoracique (largeur maximale des poumons)
        lung_bbox = lung_props[0].bbox
        thoracic_width = lung_bbox[3] - lung_bbox[1]  # xmax - xmin
        
        # Si il y a plusieurs régions pulmonaires, prendre la largeur totale
        if len(lung_props) > 1:
            all_lung_coords = []
            for prop in lung_props:
                coords = prop.coords
                all_lung_coords.extend(coords)
            all_lung_coords = np.array(all_lung_coords)
            thoracic_width = np.max(all_lung_coords[:, 1]) - np.min(all_lung_coords[:, 1])
        
        ctr = cardiac_width / thoracic_width if thoracic_width > 0 else 0
    else:
        cardiac_width = thoracic_width = ctr = 0
    
    # 3. Ratios d'aire
    lung_ratio = lung_area / total_area
    cardiac_ratio = cardiac_area / total_area
    
    # 4. Analyse de la symétrie pulmonaire
    # Séparer les poumons gauche et droit
    mid_line = w // 2
    left_lung = lung_mask[:, :mid_line]
    right_lung = lung_mask[:, mid_line:]
    
    left_area = np.sum(left_lung)
    right_area = np.sum(right_lung)
    
    # Index de symétrie (1 = parfaitement symétrique)
    if left_area + right_area > 0:
        symmetry_index = 2 * min(left_area, right_area) / (left_area + right_area)
    else:
        symmetry_index = 0
    
    # Compilation des résultats
    metrics = {
        'lung_area_pixels': lung_area,
        'cardiac_area_pixels': cardiac_area,
        'lung_ratio': lung_ratio,
        'cardiac_ratio': cardiac_ratio,
        'cardiothoracic_ratio': ctr,
        'cardiac_width': cardiac_width,
        'thoracic_width': thoracic_width,
        'left_lung_area': left_area,
        'right_lung_area': right_area,
        'lung_symmetry_index': symmetry_index
    }
    
    return metrics

# Calculer les métriques
metrics = calculate_clinical_metrics(lung_mask_gradient, cardiac_mask, img_normalized.shape)

# Affichage des résultats
print("\n" + "="*50)
print("📊 MÉTRIQUES CLINIQUES / CLINICAL METRICS")
print("="*50)

print(f"\n🫁 ANALYSE PULMONAIRE / LUNG ANALYSIS:")
print(f"   • Aire pulmonaire totale: {metrics['lung_area_pixels']} pixels")
print(f"   • Ratio pulmonaire/image: {metrics['lung_ratio']:.3f} ({metrics['lung_ratio']*100:.1f}%)")
print(f"   • Poumon gauche: {metrics['left_lung_area']} pixels")
print(f"   • Poumon droit: {metrics['right_lung_area']} pixels")
print(f"   • Index de symétrie: {metrics['lung_symmetry_index']:.3f} (1.0 = parfait)")

print(f"\n❤️ ANALYSE CARDIAQUE / CARDIAC ANALYSIS:")
print(f"   • Aire cardiaque: {metrics['cardiac_area_pixels']} pixels")
print(f"   • Ratio cardiaque/image: {metrics['cardiac_ratio']:.3f} ({metrics['cardiac_ratio']*100:.1f}%)")
print(f"   • Largeur cardiaque: {metrics['cardiac_width']:.1f} pixels")
print(f"   • Largeur thoracique: {metrics['thoracic_width']:.1f} pixels")

print(f"\n🏥 RATIO CARDIO-THORACIQUE / CARDIOTHORACIC RATIO:")
print(f"   • CTR: {metrics['cardiothoracic_ratio']:.3f}")

# Interprétation clinique
if metrics['cardiothoracic_ratio'] > 0:
    if metrics['cardiothoracic_ratio'] < 0.5:
        ctr_interpretation = "✅ Normal (< 0.5)"
    elif metrics['cardiothoracic_ratio'] < 0.6:
        ctr_interpretation = "⚠️ Limite (0.5-0.6)"
    else:
        ctr_interpretation = "🔴 Cardiomégalie possible (> 0.6)"
else:
    ctr_interpretation = "❓ Non calculable"

print(f"   • Interprétation: {ctr_interpretation}")

# Analyse de la symétrie pulmonaire
if metrics['lung_symmetry_index'] > 0.85:
    symmetry_interpretation = "✅ Symétrie normale"
elif metrics['lung_symmetry_index'] > 0.7:
    symmetry_interpretation = "⚠️ Asymétrie légère"
else:
    symmetry_interpretation = "🔴 Asymétrie significative"

print(f"\n🔍 SYMÉTRIE PULMONAIRE: {symmetry_interpretation}")

print("\n" + "="*50)

## 🎨 Visualisation Avancée / Advanced Visualization

In [ ]:
def create_comprehensive_visualization(image, lung_mask, cardiac_mask, metrics):
    """
    Création d'une visualisation complète des résultats
    Create comprehensive visualization of results
    """
    fig = plt.figure(figsize=(20, 16))
    
    # Configuration de la grille
    gs = fig.add_gridspec(3, 4, height_ratios=[2, 2, 1], hspace=0.3, wspace=0.3)
    
    # 1. Image originale
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Image Originale\nOriginal Image', fontsize=14, fontweight='bold')
    ax1.axis('off')
    
    # 2. Segmentation pulmonaire
    ax2 = fig.add_subplot(gs[0, 1])
    ax2.imshow(image, cmap='gray', alpha=0.7)
    ax2.imshow(lung_mask, cmap='Blues', alpha=0.5)
    ax2.contour(lung_mask, colors='blue', linewidths=2)
    ax2.set_title('Segmentation Pulmonaire\nLung Segmentation', fontsize=14, fontweight='bold')
    ax2.axis('off')
    
    # 3. Segmentation cardiaque
    ax3 = fig.add_subplot(gs[0, 2])
    ax3.imshow(image, cmap='gray', alpha=0.7)
    ax3.imshow(cardiac_mask, cmap='Reds', alpha=0.5)
    ax3.contour(cardiac_mask, colors='red', linewidths=2)
    ax3.set_title('Segmentation Cardiaque\nCardiac Segmentation', fontsize=14, fontweight='bold')
    ax3.axis('off')
    
    # 4. Vue d'ensemble
    ax4 = fig.add_subplot(gs[0, 3])
    ax4.imshow(image, cmap='gray')
    ax4.contour(lung_mask, colors='blue', linewidths=2, linestyles='--')
    ax4.contour(cardiac_mask, colors='red', linewidths=2)
    
    # Ajouter ligne de symétrie
    h, w = image.shape
    ax4.axvline(x=w//2, color='yellow', linestyle=':', linewidth=1, alpha=0.7)
    
    ax4.set_title('Vue d\'Ensemble\nComplete View', fontsize=14, fontweight='bold')
    ax4.axis('off')
    
    # 5. Analyse de symétrie
    ax5 = fig.add_subplot(gs[1, 0])
    
    # Séparer les poumons
    mid_line = w // 2
    left_lung = lung_mask[:, :mid_line]
    right_lung = lung_mask[:, mid_line:]
    
    # Créer une image colorée pour la symétrie
    symmetry_image = np.zeros((h, w, 3))
    symmetry_image[:, :mid_line, 0] = left_lung  # Rouge pour gauche
    symmetry_image[:, mid_line:, 2] = right_lung  # Bleu pour droite
    
    ax5.imshow(image, cmap='gray', alpha=0.3)
    ax5.imshow(symmetry_image, alpha=0.6)
    ax5.axvline(x=mid_line, color='yellow', linestyle='-', linewidth=2)
    ax5.set_title(f'Analyse de Symétrie\nSymmetry Analysis\nIndex: {metrics["lung_symmetry_index"]:.3f}', 
                 fontsize=12, fontweight='bold')
    ax5.axis('off')
    
    # 6. Graphique des aires
    ax6 = fig.add_subplot(gs[1, 1])
    
    areas = [metrics['left_lung_area'], metrics['right_lung_area'], metrics['cardiac_area_pixels']]
    labels = ['Poumon\nGauche', 'Poumon\nDroit', 'Cœur']
    colors = ['lightcoral', 'lightblue', 'lightpink']
    
    bars = ax6.bar(labels, areas, color=colors, alpha=0.7, edgecolor='black')
    ax6.set_title('Aires Segmentées\nSegmented Areas (pixels)', fontsize=12, fontweight='bold')
    ax6.set_ylabel('Aire (pixels)')
    
    # Ajouter les valeurs sur les barres
    for bar, area in zip(bars, areas):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(area)}', ha='center', va='bottom', fontweight='bold')
    
    # 7. Ratio cardio-thoracique
    ax7 = fig.add_subplot(gs[1, 2])
    
    # Graphique en secteurs pour le CTR
    if metrics['cardiothoracic_ratio'] > 0:
        ctr_value = metrics['cardiothoracic_ratio']
        remaining = 1 - ctr_value
        
        sizes = [ctr_value, remaining]
        labels_ctr = [f'Cœur\n{ctr_value:.3f}', f'Thorax\n{remaining:.3f}']
        colors_ctr = ['lightcoral', 'lightblue']
        
        wedges, texts, autotexts = ax7.pie(sizes, labels=labels_ctr, colors=colors_ctr, 
                                          autopct='%1.1f%%', startangle=90)
        ax7.set_title(f'Ratio Cardio-Thoracique\nCTR = {ctr_value:.3f}', 
                     fontsize=12, fontweight='bold')
    else:
        ax7.text(0.5, 0.5, 'CTR\nNon Calculable', ha='center', va='center', 
                transform=ax7.transAxes, fontsize=14, fontweight='bold')
        ax7.set_xlim(0, 1)
        ax7.set_ylim(0, 1)
        ax7.axis('off')
    
    # 8. Histogramme des intensités
    ax8 = fig.add_subplot(gs[1, 3])
    
    # Histogrammes par région
    lung_intensities = image[lung_mask]
    cardiac_intensities = image[cardiac_mask]
    background_intensities = image[~(lung_mask | cardiac_mask)]
    
    ax8.hist(background_intensities, bins=30, alpha=0.5, color='gray', label='Fond', density=True)
    ax8.hist(lung_intensities, bins=30, alpha=0.7, color='blue', label='Poumons', density=True)
    ax8.hist(cardiac_intensities, bins=30, alpha=0.7, color='red', label='Cœur', density=True)
    
    ax8.set_xlabel('Intensité des Pixels')
    ax8.set_ylabel('Densité')
    ax8.set_title('Distribution des Intensités\nIntensity Distribution', fontsize=12, fontweight='bold')
    ax8.legend()
    ax8.grid(True, alpha=0.3)
    
    # 9. Tableau des métriques (sur toute la largeur)
    ax9 = fig.add_subplot(gs[2, :])
    ax9.axis('off')
    
    # Données du tableau
    table_data = [
        ['Métrique / Metric', 'Valeur / Value', 'Interprétation / Interpretation'],
        ['Aire pulmonaire totale', f"{metrics['lung_area_pixels']} px ({metrics['lung_ratio']*100:.1f}%)", 
         'Surface des poumons segmentés'],
        ['Aire cardiaque', f"{metrics['cardiac_area_pixels']} px ({metrics['cardiac_ratio']*100:.1f}%)", 
         'Surface du cœur segmentée'],
        ['Ratio cardio-thoracique', f"{metrics['cardiothoracic_ratio']:.3f}", 
         'Normal < 0.5, Cardiomégalie > 0.6'],
        ['Index de symétrie', f"{metrics['lung_symmetry_index']:.3f}", 
         'Symétrie pulmonaire (1.0 = parfait)'],
        ['Poumon gauche vs droit', f"{metrics['left_lung_area']} vs {metrics['right_lung_area']} px", 
         'Comparaison des aires pulmonaires']
    ]
    
    # Créer le tableau
    table = ax9.table(cellText=table_data, cellLoc='center', loc='center',
                     colWidths=[0.3, 0.25, 0.45])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Styliser l'en-tête
    for i in range(3):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Styliser les lignes alternées
    for i in range(1, len(table_data)):
        for j in range(3):
            if i % 2 == 0:
                table[(i, j)].set_facecolor('#f0f0f0')
    
    # Titre général
    fig.suptitle('ANALYSE COMPLÈTE DE SEGMENTATION THORACIQUE\nCOMPREHENSIVE THORACIC SEGMENTATION ANALYSIS', 
                fontsize=18, fontweight='bold', y=0.95)
    
    plt.tight_layout()
    plt.show()

# Créer la visualisation complète
create_comprehensive_visualization(img_normalized, lung_mask_gradient, cardiac_mask, metrics)

print("✅ Visualisation complète générée / Complete visualization generated")

## 🏥 Interprétation Clinique / Clinical Interpretation

### Contexte pour les Professionnels de Santé / Context for Healthcare Professionals

In [ ]:
def generate_clinical_report(metrics):
    """
    Génération d'un rapport clinique automatique
    Generate automatic clinical report
    """
    print("📋 RAPPORT CLINIQUE AUTOMATIQUE")
    print("📋 AUTOMATIC CLINICAL REPORT")
    print("="*60)
    
    # 1. Résumé exécutif
    print("\n🎯 RÉSUMÉ EXÉCUTIF / EXECUTIVE SUMMARY:")
    print("-" * 40)
    
    # Analyse du CTR
    ctr = metrics['cardiothoracic_ratio']
    if ctr > 0:
        if ctr < 0.5:
            ctr_status = "NORMAL"
            ctr_color = "🟢"
        elif ctr < 0.6:
            ctr_status = "LIMITE"
            ctr_color = "🟡"
        else:
            ctr_status = "ANORMAL"
            ctr_color = "🔴"
        
        print(f"   {ctr_color} Ratio Cardio-Thoracique: {ctr:.3f} - {ctr_status}")
    else:
        print("   ❓ Ratio Cardio-Thoracique: Non calculable")
    
    # Analyse de la symétrie
    symmetry = metrics['lung_symmetry_index']
    if symmetry > 0.85:
        sym_status = "NORMALE"
        sym_color = "🟢"
    elif symmetry > 0.7:
        sym_status = "LÉGÈREMENT ASYMÉTRIQUE"
        sym_color = "🟡"
    else:
        sym_status = "ASYMÉTRIQUE"
        sym_color = "🔴"
    
    print(f"   {sym_color} Symétrie Pulmonaire: {symmetry:.3f} - {sym_status}")
    
    # 2. Pour les chirurgiens
    print("\n🏥 POUR LES CHIRURGIENS / FOR SURGEONS:")
    print("-" * 40)
    
    print("   📐 Planification pré-opératoire:")
    print(f"      • Volume pulmonaire segmenté: {metrics['lung_ratio']*100:.1f}% de l'image")
    print(f"      • Asymétrie pulmonaire: {abs(metrics['left_lung_area'] - metrics['right_lung_area'])} pixels")
    
    if metrics['lung_symmetry_index'] < 0.8:
        print("      ⚠️ Attention à l'asymétrie pulmonaire lors de l'intervention")
    
    print("   🫁 Considérations chirurgicales:")
    if metrics['left_lung_area'] > metrics['right_lung_area'] * 1.2:
        print("      • Prédominance du poumon gauche")
    elif metrics['right_lung_area'] > metrics['left_lung_area'] * 1.2:
        print("      • Prédominance du poumon droit")
    else:
        print("      • Volumes pulmonaires équilibrés")
    
    # 3. Pour les médecins généralistes
    print("\n👨‍⚕️ POUR LES MÉDECINS GÉNÉRALISTES / FOR GPs:")
    print("-" * 40)
    
    print("   🔍 Aide au diagnostic:")
    
    if ctr > 0.6:
        print("      🔴 ALERTE: Possible cardiomégalie détectée")
        print("      → Recommandation: Échocardiographie et consultation cardiologique")
    elif ctr > 0.5:
        print("      🟡 Surveillance: CTR à la limite supérieure")
        print("      → Recommandation: Suivi clinique et radiologique")
    else:
        print("      🟢 CTR dans les limites normales")
    
    if symmetry < 0.7:
        print("      🔴 ATTENTION: Asymétrie pulmonaire significative")
        print("      → Recommandation: TDM thoracique pour exploration")
    
    print("   📋 Orientation diagnostique:")
    if metrics['lung_ratio'] < 0.3:  # Poumons occupent moins de 30% de l'image
        print("      • Volumes pulmonaires réduits - Explorer: fibrose, épanchement")
    elif metrics['lung_ratio'] > 0.6:  # Poumons occupent plus de 60% de l'image
        print("      • Volumes pulmonaires augmentés - Explorer: emphysème, BPCO")
    
    # 4. Pour les enseignants d'anatomie
    print("\n📚 POUR LES ENSEIGNANTS D'ANATOMIE / FOR ANATOMY TEACHERS:")
    print("-" * 40)
    
    print("   🎓 Points pédagogiques:")
    print(f"      • Identification automatique des structures thoraciques")
    print(f"      • Quantification objective des rapports anatomiques")
    print(f"      • CTR = {ctr:.3f} (largeur cardiaque / largeur thoracique)")
    
    print("   📖 Corrélations anatomiques:")
    print("      • Silhouette cardiaque: oreillette droite, ventricules, arc aortique")
    print("      • Champs pulmonaires: parenchyme, scissures, angles costophréniques")
    print("      • Symétrie: anatomie comparative des lobes pulmonaires")
    
    # 5. Métriques techniques
    print("\n🔬 MÉTRIQUES TECHNIQUES / TECHNICAL METRICS:")
    print("-" * 40)
    
    print(f"   • Aire pulmonaire: {metrics['lung_area_pixels']} pixels")
    print(f"   • Aire cardiaque: {metrics['cardiac_area_pixels']} pixels")
    print(f"   • Poumon gauche: {metrics['left_lung_area']} pixels")
    print(f"   • Poumon droit: {metrics['right_lung_area']} pixels")
    print(f"   • Largeur cardiaque: {metrics['cardiac_width']:.1f} pixels")
    print(f"   • Largeur thoracique: {metrics['thoracic_width']:.1f} pixels")
    
    # 6. Limitations et recommandations
    print("\n⚠️ LIMITATIONS ET RECOMMANDATIONS:")
    print("-" * 40)
    print("   • Cette analyse est automatique et doit être validée cliniquement")
    print("   • La qualité de segmentation dépend de la qualité de l'image")
    print("   • Les mesures sont en pixels, pas en unités anatomiques absolues")
    print("   • Toujours corréler avec l'examen clinique et l'anamnèse")
    
    print("\n" + "="*60)
    print("📊 Rapport généré automatiquement par TorchXRayVision")
    print("📊 Report automatically generated by TorchXRayVision")
    print("="*60)

# Générer le rapport clinique
generate_clinical_report(metrics)

## 🎯 Conclusion et Perspectives / Conclusion and Perspectives

### Ce que nous avons appris / What we learned

Dans ce tutoriel, nous avons exploré :

1. **Segmentation par intensité** : Méthode simple basée sur les valeurs de pixels
2. **Segmentation par gradients** : Approche plus sophistiquée utilisant les contours
3. **Segmentation multi-structures** : Identification simultanée des poumons et du cœur
4. **Analyse quantitative** : Calcul de métriques cliniques importantes
5. **Interprétation clinique** : Application pratique en médecine

### Applications cliniques / Clinical applications

- **Diagnostic assisté** : Aide à la détection d'anomalies
- **Planification chirurgicale** : Évaluation pré-opératoire
- **Suivi longitudinal** : Évolution des pathologies
- **Enseignement médical** : Formation interactive

### Prochaines étapes / Next steps

Le **Tutorial 4** portera sur :
- Détection et localisation de pathologies
- Classification des anomalies pulmonaires
- Cartographie des lésions
- Rapport automatique d'analyse

## 💾 Sauvegarde des Résultats / Save Results

Sauvegardons nos résultats pour le prochain tutoriel :

In [ ]:
# Sauvegarder les résultats de segmentation
import pickle

# Préparer les données à sauvegarder
segmentation_results = {
    'original_image': img_normalized,
    'lung_mask': lung_mask_gradient,
    'cardiac_mask': cardiac_mask,
    'metrics': metrics,
    'image_shape': img_normalized.shape
}

# Sauvegarder en format pickle
with open('segmentation_results_tutorial3.pkl', 'wb') as f:
    pickle.dump(segmentation_results, f)

print("💾 Résultats sauvegardés dans 'segmentation_results_tutorial3.pkl'")
print("💾 Results saved to 'segmentation_results_tutorial3.pkl'")

# Sauvegarder aussi les images pour visualisation
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(img_normalized, cmap='gray')
plt.title('Image Originale')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(img_normalized, cmap='gray')
plt.contour(lung_mask_gradient, colors='blue', linewidths=2)
plt.title('Segmentation Pulmonaire')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(img_normalized, cmap='gray')
plt.contour(lung_mask_gradient, colors='blue', linewidths=2)
plt.contour(cardiac_mask, colors='red', linewidths=2)
plt.title('Segmentation Complète')
plt.axis('off')

plt.tight_layout()
plt.savefig('segmentation_summary_tutorial3.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Tutorial 3 terminé avec succès!")
print("✅ Tutorial 3 completed successfully!")
print("\n🎯 Prêt pour le Tutorial 4: Pathology Detection and Localization")
print("🎯 Ready for Tutorial 4: Pathology Detection and Localization")